In [1]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_url = "mysql+mysqlconnector://{USER}:{PWD}@{HOST}/{DBNAME}"

db_url = db_url.format(
    USER = "root", 
    PWD = "QsLdzoV7u9ZxuIeL",
    HOST = "localhost:3306",
    DBNAME = "eventfrogdata"
)

In [3]:
sql_query = """
SELECT 
    `web-scraper-order`,
    name,
    description,
    price,
    address,
    category,
    time,
    MAKEDATE(2023, 334 + SUBSTRING(date, 5, 3)) AS date_formatted
FROM
    events_raw
WHERE
    is_eventgruppe IS NULL
ORDER BY date_formatted ASC;
"""

In [4]:
engine = create_engine(db_url, echo=False)

with engine.begin() as conn:
    
    df_events = pd.read_sql_query(
        sql=sql_query, # query
        con=conn # database connection 
    )

In [5]:
print(df_events.shape)
df_events.head()

(1671, 8)


,web-scraper-order,name,description,price,address,category,time,date_formatted
0,1701178304-1235,Live-Musik mit Sina Anastasia,Ihr Name *ein Hauch von Melodie*\n Ihre Ausstr...,None,"Kellertheater im Haus der Vereine, Baselstrass...",Konzert / Singer & Songwriter,19:00 Uhr\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\...,2023-12-01
1,1701178306-1236,"Metal Marmot, 1. - 2. Dezember 2023",Metal Marmot im Dezember 2023: Freitag 1. Deze...,CHF 40.00,"Hotel Simplon, Kanderstegstrasse 25, Frutigen, CH",Konzert / Hard Rock & Heavy Metal,Beginn:\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t1...,2023-12-01
2,1701178308-1237,JAMfreeday,Alle sind herzlich eingeladen um zu musizieren...,None,"Dä 3.Stock, Industriestrasse, 28, Herisau, CH",Konzert,19:00 Uhr,2023-12-01
3,1701178310-1238,Am Hummelwald,"Der Kurzprosaband ""Am Hummelwald - Miniaturen ...",None,"Raum für Literatur, St.Leonhard-Strasse 40, St...",Konzert,19:00 Uhr,2023-12-01
4,1701178312-1239,BELPHEGOR 30 Year Tour Special Guest: Arkona &...,BELPHEGOR MARSCHIERT DURCH EUROPA WINTER MMXXI...,CHF 38.00,"Musigburg, Aarburg, CH",Konzert / Hard Rock & Heavy Metal,19:00 Uhr\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\...,2023-12-01


In [6]:
df_events['price'] = df_events['price'].str.extract('(\d+.\d+)').astype(float)
df_events['price']

0        NaN
1       40.0
2        NaN
3        NaN
4       38.0
        ... 
1666     NaN
1667     NaN
1668     NaN
1669     NaN
1670     NaN
Name: price, Length: 1671, dtype: float64

In [7]:
df_events._get_value(1, 'time')

'Beginn:\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t19:00 Uhr'

In [8]:
df_time = df_events['time'].str.extractall('(\d+:\d+)').unstack()

In [9]:
df_events['time_from'] = df_time[0][0]
df_events['time_until'] = df_time[0][1]
df_events = df_events.drop(columns='time')
df_events.head()

,web-scraper-order,name,description,price,address,category,date_formatted,time_from,time_until
0,1701178304-1235,Live-Musik mit Sina Anastasia,Ihr Name *ein Hauch von Melodie*\n Ihre Ausstr...,NaN,"Kellertheater im Haus der Vereine, Baselstrass...",Konzert / Singer & Songwriter,2023-12-01,19:00,22:00
1,1701178306-1236,"Metal Marmot, 1. - 2. Dezember 2023",Metal Marmot im Dezember 2023: Freitag 1. Deze...,40.0,"Hotel Simplon, Kanderstegstrasse 25, Frutigen, CH",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,NaN
2,1701178308-1237,JAMfreeday,Alle sind herzlich eingeladen um zu musizieren...,NaN,"Dä 3.Stock, Industriestrasse, 28, Herisau, CH",Konzert,2023-12-01,19:00,NaN
3,1701178310-1238,Am Hummelwald,"Der Kurzprosaband ""Am Hummelwald - Miniaturen ...",NaN,"Raum für Literatur, St.Leonhard-Strasse 40, St...",Konzert,2023-12-01,19:00,NaN
4,1701178312-1239,BELPHEGOR 30 Year Tour Special Guest: Arkona &...,BELPHEGOR MARSCHIERT DURCH EUROPA WINTER MMXXI...,38.0,"Musigburg, Aarburg, CH",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,23:50


In [10]:
df_events['duration_seconds'] = pd.to_datetime(df_events['time_until'], format='%H:%M') - pd.to_datetime(df_events['time_from'], format='%H:%M')
df_events['duration_seconds'] = df_events['duration_seconds'].dt.seconds

In [11]:
df_events['is_concert'] = df_events['category'].str.contains('Konzert').astype(int)

In [12]:
df_events['description'] = df_events['description'].replace(r'\n',' ', regex=True)
df_events['description'] = df_events['description'].str.lower()
df_events['description_len'] = df_events['description'].str.len()

In [14]:
df_events.to_csv('data/events_prepared.csv')